In [22]:
import numpy as np
import faiss
from faiss.contrib.datasets import SyntheticDataset

This script shows how to simulate an IndexPQ on a GPU with a 1-centroid IVFPQ.

In [23]:
ds = SyntheticDataset(64, 10000, 5000, 1000) # 64-d vectors, 10k train, 5k queries, 1k ground truth

In [24]:
index_cpu = faiss.index_factory(ds.d, "OPQ8,PQ4x8np")

In [25]:
index_cpu

<faiss.swigfaiss_avx2.IndexPreTransform; proxy of <Swig Object of type 'faiss::IndexPreTransform *' at 0x7efc324e9470> >

In [26]:
index_cpu.chain.at(0)

<faiss.swigfaiss_avx2.VectorTransform; proxy of <Swig Object of type 'faiss::VectorTransform *' at 0x7efc3253c4b0> >

In [27]:
index_cpu.index

<faiss.swigfaiss_avx2.Index; proxy of <Swig Object of type 'faiss::Index *' at 0x7efc3253d830> >

In [28]:
faiss.downcast_index(index_cpu.index)

<faiss.swigfaiss_avx2.IndexPQ; proxy of <Swig Object of type 'faiss::IndexPQ *' at 0x7efc3253dbf0> >

In [29]:
idx = faiss.index_factory(ds.d, "IVF4096,PQ4x8np")

In [30]:
idx

<faiss.swigfaiss_avx2.IndexIVFPQ; proxy of <Swig Object of type 'faiss::IndexIVFPQ *' at 0x7efc9e495ef0> >

In [31]:
index_cpu.train(ds.get_train())

In [32]:
index_cpu.add(ds.get_database())

In [33]:
%%time
Dref, Iref = index_cpu.search(ds.get_queries(), 10)

CPU times: user 1.04 s, sys: 22.6 ms, total: 1.07 s
Wall time: 8.96 ms
Parser   : 135 ms


# Copy to IVF index

In [34]:
index_pq = faiss.downcast_index(index_cpu.index)
index_to_translate = faiss.index_factory(index_pq.d, "IVF1,PQ4x8") #IndexPQ(dim, M, nbits, METRIC_INNER_PRODUCT) with M=4, nbits=8
index_to_translate.quantizer.add(np.zeros((1, index_pq.d)))
index_to_translate.pq = index_pq.pq
index_to_translate.is_trained = True

In [35]:
index_with_transform = faiss.IndexPreTransform(index_cpu.chain.at(0), index_to_translate)
index_with_transform.add(ds.get_database())  # this is not necessary, only for testing 

In [36]:
index_cpu.index

<faiss.swigfaiss_avx2.Index; proxy of <Swig Object of type 'faiss::Index *' at 0x7efc3253de90> >

In [37]:
index_to_translate

<faiss.swigfaiss_avx2.IndexIVFPQ; proxy of <Swig Object of type 'faiss::IndexIVFPQ *' at 0x7efc8c1d6b80> >

In [38]:
%%time
D2, I2 = index_with_transform.search(ds.get_queries(), 10)

CPU times: user 2.2 s, sys: 109 ms, total: 2.31 s
Wall time: 22.5 ms


In [39]:
np.testing.assert_array_equal(I2, Iref)

In [40]:
index_with_transform.reset()

# Copy to GPU index

In [ ]:
#

In [41]:
res = faiss.StandardGpuResources()

In [42]:
index_gpu = faiss.index_cpu_to_gpu(res, 0, index_to_translate)

In [43]:
index_gpu_with_transform = faiss.IndexPreTransform(index_cpu.chain.at(0), index_gpu)

In [47]:
index_gpu_with_transform.chain.at(0)

<faiss.swigfaiss_avx2.VectorTransform; proxy of <Swig Object of type 'faiss::VectorTransform *' at 0x7efc32427750> >

In [49]:
faiss.downcast_index(index_gpu_with_transform.index)

<faiss.swigfaiss_avx2.GpuIndexIVFPQ; proxy of <Swig Object of type 'faiss::gpu::GpuIndexIVFPQ *' at 0x7efc32427090> >

In [44]:
index_gpu_with_transform.add(ds.get_database())

In [50]:
n = 0
while n < 100000:
    D3, I3 = index_gpu_with_transform.search(ds.get_queries(), 10)
    n += 1

In [46]:
np.testing.assert_array_equal(I3, Iref)

AssertionError: 
Arrays are not equal

Mismatched elements: 4 / 10000 (0.04%)
Max absolute difference: 4088
Max relative difference: 10.50899743
 x: array([[1985, 2261, 1879, ..., 3369, 4329, 1409],
       [3937, 1765, 4759, ...,  443, 1996, 2927],
       [3108, 1395,  525, ...,  934, 4571,  125],...
 y: array([[1985, 2261, 1879, ..., 3369, 4329, 1409],
       [3937, 1765, 4759, ...,  443, 1996, 2927],
       [3108, 1395,  525, ...,  934, 4571,  125],...

# Without OPQ

In [51]:
import numpy as np
import faiss
from faiss.contrib.datasets import SyntheticDataset

ds = SyntheticDataset(64, 10000, 5000, 1000) # 64-d vectors, 10k train, 5k queries, 1k ground truth

In [62]:
M = 4
nbits = 8

index_cpu = faiss.index_factory(ds.d, f"PQ{M}x{nbits}", faiss.METRIC_INNER_PRODUCT)
index_cpu.train(ds.get_train())

# copy to ivf
index_ivf = faiss.index_factory(index_cpu.d, f"IVF1,PQ{M}x{nbits}", faiss.METRIC_INNER_PRODUCT)
index_ivf.quantizer.add(np.zeros((1, index_pq.d)))
index_ivf.pq = index_cpu.pq
index_ivf.is_trained = True

In [63]:
# copy to gpu
res = faiss.StandardGpuResources()
index_ivf_gpu = faiss.index_cpu_to_gpu(res, 0, index_ivf)

In [64]:
# compare
index_cpu.add(ds.get_database())
index_ivf.add(ds.get_database())
index_ivf_gpu.add(ds.get_database())

D1, I1 = index_cpu.search(ds.get_queries(), 10)
D2, I2 = index_ivf.search(ds.get_queries(), 10)
D3, I3 = index_ivf_gpu.search(ds.get_queries(), 10)

In [65]:
np.testing.assert_array_equal(D1, D2)

AssertionError: 
Arrays are not equal

Mismatched elements: 4527 / 10000 (45.3%)
Max absolute difference: 2.861023e-06
Max relative difference: 2.3387753e-07
 x: array([[11.379868, 10.36752 , 10.255153, ...,  9.338544,  9.169638,
         9.015741],
       [10.261639,  9.711093,  9.247979, ...,  8.732635,  8.291955,...
 y: array([[11.379868, 10.367521, 10.255154, ...,  9.338543,  9.169637,
         9.015741],
       [10.261639,  9.711092,  9.24798 , ...,  8.732635,  8.291955,...

In [66]:
np.testing.assert_array_equal(D1, D3)

In [61]:
# assertion fails because of floating point errors